<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=347785d2550303aaf19a550431900abe86c7b9e0eb8e5614781596a603319aa3
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-20 15:22:43
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.36 C
-------------------
Today PnL: -3.29 L (-2.37%)
Current PnL: -33.27 L (-21.22%)
CY Booked + Current PnL: -18.98 L (-12.11%)
-------------------
Total profit:  1.25 L
Total loss:  -34.52 L
-------------------
Total Booked + Current PnL: 8.00 L (6.16%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.52%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 98.63 L (72.63%)
Deployed:  1.30 C
Current:  1.36 C
CAGR/XIRR %: 2.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-1.53,14.21,6.29,21.40,19068.0,37721.0,303147.0,5.95,53.0,X-LC,6.88,14.0,1.98,2.23,21.44,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.84,-23.20,30.32,0.09,23468.0,-23376.0,77401.0,79.72,37.0,M-SC,4.71,253.0,-1.00,0.57,1.84,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-3.66,1.91,17.00,19.23,26066.0,2868.0,153328.0,19.88,51.0,L-LC,10.34,182.0,0.11,1.13,23.59,XY25,ATH,MINING
49,MASFIN,398.61,-2.30,-7.10,31.38,22.05,28562.0,-6960.0,91020.0,-20.05,40.0,H-SC,4.94,168.0,-0.24,0.67,32.30,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.91,-19.10,23.64,0.02,33382.0,-33348.0,141211.0,115.40,41.0,M-SC,9.09,240.0,-1.00,1.04,22.83,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HINDUNILVR,2922.0,2.27,-6.10,21.05,13.67,50813.0,-15673.0,241390.0,-13.11,60.0,X-LC,2.56,9.0,-0.31,1.78,13.96,XY25,NTT,FMCG
56,RAJOOENG,143.1,2.06,-31.90,105.01,39.61,73297.0,-32700.0,69800.0,-53.92,54.0,H-SC,29.00,136.0,-0.45,0.51,12.98,AR,ATH,MISC
77,TTKPRESTIG,770.0,1.84,-23.20,30.32,0.09,23468.0,-23376.0,77401.0,79.72,37.0,M-SC,4.71,253.0,-1.00,0.57,1.84,OX40N,NTT,DURABLES
14,CAMPUS,393.0,0.54,-15.16,57.11,33.28,81288.0,-25440.0,142335.0,-31.68,35.0,M-SC,4.55,221.0,-0.31,1.05,10.32,XY24,NTT,FOOTWEAR
28,GREENPANEL,537.0,-0.09,-43.89,155.08,43.14,159320.0,-80344.0,102734.0,129.97,27.0,M-SC,14.72,231.0,-0.50,0.76,0.00,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-7.50,-14.30,54.59,32.48,153444.0,-46906.0,281085.0,-18.59,32.0,X-MC,5.15,69.0,-0.31,2.07,0.00,X40,ATH,ELECTRICAL
0,5PAISA,593.00,-6.41,-32.48,67.21,12.90,99158.0,-70962.0,147534.0,117.55,50.0,H-SC,7.82,173.0,-0.72,1.09,22.67,OX40N,NTT,FINANCE
82,VOLTAS,1856.00,-5.36,3.54,40.23,45.20,79867.0,6783.0,198525.0,-6.60,33.0,X-MC,5.67,80.0,0.08,1.46,10.58,XY25,NTT,AC
40,INDUSINDBK,1800.00,-5.11,-33.46,99.72,32.89,51227.0,-25836.0,51371.0,-734.18,53.0,L-MC,10.18,259.0,-0.50,0.38,41.47,XR,NTT,BANKS
37,IEX,219.00,-5.03,-11.70,68.32,48.63,120625.0,-23395.0,176559.0,-41.22,38.0,H-SC,13.25,137.0,-0.19,1.30,0.00,XR,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-3.36,-7.45,124.64,107.91,170321.0,-10994.0,136650.0,-24.60,37.0,M-SC,9.94,220.0,-0.06,1.01,1.89,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.91,-19.10,23.64,0.02,33382.0,-33348.0,141211.0,115.40,41.0,M-SC,9.09,240.0,-1.00,1.04,22.83,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-3.25,-22.43,70.22,32.03,71806.0,-29577.0,102258.0,-16.04,48.0,H-SC,6.39,157.0,-0.41,0.75,14.54,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.84,-23.20,30.32,0.09,23468.0,-23376.0,77401.0,79.72,37.0,M-SC,4.71,253.0,-1.00,0.57,1.84,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-3.15,-24.77,50.84,13.47,103134.0,-66807.0,202860.0,-69.09,40.0,H-SC,4.24,159.0,-0.65,1.49,3.15,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-3.66,1.91,17.0,19.23,26066.0,2868.0,153328.0,19.88,51.0,L-LC,10.34,182.0,0.11,1.13,23.59,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.42,-1.56,75.00,72.27,125100.0,-2648.0,166800.0,-15.61,32.0,M-LC,5.10,101.0,-0.02,1.23,4.52,XR,NTT,IT
25,FINCABLES,1641.55,-3.36,-7.45,124.64,107.91,170321.0,-10994.0,136650.0,-24.60,37.0,M-SC,9.94,220.0,-0.06,1.01,1.89,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.45,-8.01,124.72,106.72,141504.0,-9879.0,113457.0,-54.44,43.0,H-SC,8.64,138.0,-0.07,0.84,13.50,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.95,-7.97,47.38,35.63,91380.0,-16707.0,192867.0,-20.73,34.0,H-MC,4.51,119.0,-0.18,1.42,7.37,AR,ATH,PHARMA
37,IEX,219.00,-5.03,-11.70,68.32,48.63,120625.0,-23395.0,176559.0,-41.22,38.0,H-SC,13.25,137.0,-0.19,1.30,0.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.95,-31.93,130.94,57.19,290662.0,-104146.0,221981.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
43,ITC,452.00,-2.13,-20.08,38.61,10.78,74159.0,-48245.0,192073.0,-52.19,16.0,X-LC,11.79,1.0,-0.65,1.41,0.0,X40,NTT,FMCG
58,RELAXO,1176.00,-3.56,-52.97,223.88,52.31,152827.0,-76897.0,68263.0,-49.96,18.0,X-SC,8.92,92.0,-0.50,0.50,0.0,X40N,NTT,FOOTWEAR
53,PGHH,17907.65,-1.73,-10.44,49.83,34.19,95291.0,-22288.0,191232.0,-37.07,21.0,X-MC,5.13,60.0,-0.23,1.41,0.0,X40,ATH,FMCG
52,PAGEIND,51605.07,-3.44,-16.21,55.88,30.62,110994.0,-38424.0,198630.0,-40.42,22.0,X-MC,17.40,57.0,-0.35,1.46,0.0,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-2.14,-14.87,39.26,18.55,126411.0,-56253.0,321984.0,-26.54,42.0,X-LC,1.20,3.0,-0.45,2.37,9.54,X40,ATH,IT
20,DABUR,735.00,-1.60,-1.35,45.67,43.70,111758.0,-3361.0,244707.0,-9.42,44.0,X-MC,2.07,73.0,-0.03,1.80,13.49,XY24,BTT,FMCG
45,JIOFIN,387.00,-3.99,-14.87,46.23,24.48,105219.0,-39766.0,227599.0,-16.28,25.0,X-LC,2.16,37.0,-0.38,1.68,31.92,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-0.64,-3.31,24.90,20.77,56375.0,-7740.0,226404.0,-11.68,33.0,X-MC,2.23,70.0,-0.14,1.67,3.38,X40N,NTT,BREWERIES
35,ICICIGI,2252.93,-1.30,-1.46,21.11,19.34,41627.0,-2917.0,197192.0,-22.15,35.0,X-MC,2.42,71.0,-0.07,1.45,14.25,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-2.31,-28.83,130.63,64.15,221235.0,-68591.0,169360.0,-58.50,37.0,X-SC,6.69,84.0,-0.31,1.25,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-0.95,-31.93,130.94,57.19,290662.0,-104146.0,221981.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
9,BAJAJHLDNG,14451.52,-0.80,-5.55,36.81,29.22,66100.0,-10554.0,179571.0,-8.00,32.0,X-LC,12.23,29.0,-0.16,1.32,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.67,-23.06,102.07,55.47,191060.0,-56101.0,187185.0,-31.90,24.0,X-MC,16.47,63.0,-0.29,1.38,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.53,-42.59,135.52,35.20,100103.0,-54804.0,73866.0,-3.10,22.0,X-SC,17.89,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-3.04,-32.96,113.07,42.84,49277.0,-21425.0,43581.0,-55.92,34.0,X-SC,23.37,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-3.56,-52.97,223.88,52.31,152827.0,-76897.0,68263.0,-49.96,18.0,X-SC,8.92,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.53,-42.59,135.52,35.20,100103.0,-54804.0,73866.0,-3.10,22.0,X-SC,17.89,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.01,-9.76,37.49,24.08,42950.0,-12385.0,114565.0,-17.04,31.0,X-SC,6.42,91.0,-0.29,0.84,7.95,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-1.13,-18.12,75.04,43.32,100073.0,-29512.0,133360.0,-29.66,44.0,X-SC,8.18,90.0,-0.29,0.98,2.40,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-2.13,-20.08,38.61,10.78,74159.0,-48245.0,192073.0,-52.19,16.0,X-LC,11.79,1.0,-0.65,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.74,-30.72,77.51,22.98,121298.0,-69404.0,156494.0,-27.96,31.0,X-LC,3.61,2.0,-0.57,1.15,0.00,XY24,NTT,AUTO
73,TCS,4311.59,-2.14,-14.87,39.26,18.55,126411.0,-56253.0,321984.0,-26.54,42.0,X-LC,1.20,3.0,-0.45,2.37,9.54,X40,ATH,IT
81,VBL,671.64,-1.74,-1.56,37.81,35.67,117562.0,-4913.0,310929.0,-13.27,48.0,X-LC,2.94,5.0,-0.04,2.29,12.08,X40N,ATH,FMCG
41,INFY,1972.00,-1.56,12.66,19.16,34.24,67538.0,39606.0,352494.0,-12.42,57.0,X-LC,7.37,6.0,0.59,2.60,21.99,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-5.11,-33.46,99.72,32.89,51227.0,-25836.0,51371.0,-734.18,53.0,L-MC,10.18,259.0,-0.50,0.38,41.47,XR,NTT,BANKS
17,COALINDIA,484.83,-3.66,1.91,17.00,19.23,26066.0,2868.0,153328.0,19.88,51.0,L-LC,10.34,182.0,0.11,1.13,23.59,XY25,ATH,MINING
5,ASIANPAINT,3460.25,-2.91,-6.54,29.41,20.95,69200.0,-16474.0,235294.0,-7.52,37.0,X-LC,12.73,36.0,-0.24,1.73,26.76,X40,ATH,PAINTS
32,HCLTECH,1908.19,-1.87,9.33,13.27,23.83,35099.0,22574.0,264498.0,14.03,59.0,X-LC,6.34,13.0,0.64,1.95,26.90,X40,ATH,IT
51,NESTLEIND,1377.00,-1.53,14.21,6.29,21.40,19068.0,37721.0,303147.0,5.95,53.0,X-LC,6.88,14.0,1.98,2.23,21.44,XY25,NTT,FMCG


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-6.41,-32.48,67.21,12.90,99158.0,-70962.0,147534.0,117.55,50.0,H-SC,7.82,173.0,-0.72,1.09,22.67,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,2.06,-31.90,105.01,39.61,73297.0,-32700.0,69800.0,-53.92,54.0,H-SC,29.00,136.0,-0.45,0.51,12.98,AR,ATH,MISC
17,COALINDIA,484.83,-3.66,1.91,17.00,19.23,26066.0,2868.0,153328.0,19.88,51.0,L-LC,10.34,182.0,0.11,1.13,23.59,XY25,ATH,MINING
62,SATIN,274.00,-1.59,-17.39,83.81,51.83,191276.0,-48058.0,228226.0,-26.41,56.0,H-SC,3.59,148.0,-0.25,1.68,11.18,XY24,NTT,FINANCE
40,INDUSINDBK,1800.00,-5.11,-33.46,99.72,32.89,51227.0,-25836.0,51371.0,-734.18,53.0,L-MC,10.18,259.0,-0.50,0.38,41.47,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.62
1,25,44.86
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.46
MC    30.65
LC    26.86
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.22
X40      24.16
X40N     12.91
XY25     10.93
XR        8.75
AR        8.42
OX40N     7.64
SR        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
X-LC    23.45
H-SC    22.69
M-SC    11.09
X-SC     7.90
H-MC     4.54
M-MC     1.45
M-LC     1.23
L-LC     1.13
H-LC     1.05
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.48,-10.94,47.34
IT,12.85,-22.54,83.76
FINANCE,11.05,-18.87,68.76
MISC,7.04,-35.73,91.25
PAINTS,5.80,-17.18,34.59
ELECTRICAL,5.75,-20.12,64.12
INSURANCE,4.76,-4.93,40.84
PHARMA,3.96,-9.19,43.56
BANKS,2.77,-42.10,128.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3408628.0,21
AR,1427768.0,10
XR,1303648.0,12
X40,1227286.0,15
X40N,990074.0,9
OX40N,755255.0,10
XY25,458076.0,7
SR,292038.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3710416.0,24
X-MC,1736359.0,16
M-SC,1553643.0,15
X-LC,1028783.0,13
X-SC,844880.0,8
H-MC,461584.0,3
L-SC,184862.0,2
M-LC,125100.0,1
H-LC,91279.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1298752.0      6
           AR         951440.0      5
M-SC       XY24       886638.0      6
H-SC       XR         842098.0      7
X-MC       X40        589769.0      7
           XY24       469247.0      3
X-LC       X40        438507.0      6
X-MC       X40N       436066.0      4
M-SC       OX40N      335199.0      5
X-SC       X40N       332408.0      3
H-SC       OX40N      326088.0      4
X-SC       XY24       313462.0      3
H-SC       SR         292038.0      2
H-MC       AR         247572.0      2
X-MC       XY25       241277.0      2
X-LC       XY24       226517.0      2
           X40N       221600.0      2
H-MC       XY24       214012.0      1
X-SC       X40        199010.0      2
M-SC       XR         194329.0      2
X-LC       XY25       142159.0      3
M-SC       AR         137477.0      2
M-LC       XR         125100.0      1
L-SC       OX40N       93968.0      1
H-LC       AR          91279.0      1
L-SC       XR          90894.0      1
L-MC       XR          51227.0      1
M-MC       XY25        48574.0      1
L-LC       XY25        26066.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
